In [1]:
import os
import xml.dom.minidom
import xml.etree.ElementTree as ET
from PIL import Image

### verify the integrity of images for test

In [2]:
import os
import shutil
from PIL import Image

test_path = "../../yolo_sc_20181128/test"
wrong_path = "../../yolo_sc_20181128/wrong"
os.makedirs(wrong_path, exist_ok=True)

img_names = [os.path.join(test_path, f) for f in os.listdir(test_path) if f.endswith(".jpg")]
wrong = 0
for img_name in img_names:
    try:
        img = Image.open(img_name)
        img.close()
    except:
        wrong += 1
        shutil.move(img_name, wrong_path)
#         print("image corrupted", img_name)
print("total: {}, wrong: {}".format(len(img_names), wrong))

total: 4298, wrong: 0


### pad test images to target sizes (608x608)
#### image at its original size, usually much smaller, is not good for direct yolov3 prediction

In [3]:
import os
from PIL import Image

def resize_img_with_padding(img_name, size, save_path):
    img_name_new = os.path.join(save_path, os.path.basename(img_name))
    txt_name_new = os.path.splitext(img_name_new)[0] + ".txt"
    
    img = Image.open(img_name)
    
    # pad and crop image
    x = -((size - img.size[0]) // 2)
    y = -((size - img.size[1]) // 2)
    img_croped = img.crop(
        (
            x,
            y,
            size + x,
            size + y
        )
    )
    img_croped.save(img_name_new)
        
    # write lables to txt
    with open(txt_name_new, 'w') as f:
        f.write("padx {} pady {}\n".format(x, y))
            
#     print("processed ", img_name)

In [4]:
test_path = "../../yolo_sc_20181128/test"
test608_path = "../../yolo_sc_20181128/test608"
size = 608

os.makedirs(test608_path, exist_ok=True)
img_names = [os.path.join(test_path, f) for f in os.listdir(test_path) if f.endswith(".jpg")]
for img_name in img_names:
    resize_img_with_padding(img_name, size, save_path=test608_path)

### generate xmls from prediction, jpg is not padded to target size

In [ ]:
def collect_labels(result_txt, class_i):
    all_labels = {}
    with open(result_txt, 'r') as f:
        for line in f.readlines():
            tokens = line.strip().rsplit(' ', 5)  # split from right, in case there is blank space in jpg name
            basename = tokens[0]  # without file extension
            det = float(tokens[1])
            xmin, ymin = int(float(tokens[2])), int(float(tokens[3]))
            xmax, ymax = int(float(tokens[4])), int(float(tokens[5]))
            if not basename in all_labels:
                all_labels[basename] = []
            all_labels[basename].append({"det":det, "xmin":xmin, "ymin":ymin, "xmax":xmax, "ymax":ymax, "name":class_i})
    return all_labels


def generate_xml(basename, labels, data_path, save_path):
    jpg_name = os.path.join(data_path, basename+".jpg")
    with Image.open(jpg_name) as img:
        w, h = img.size
    
    root = ET.Element("annotation")
    ET.SubElement(root, "folder").text = "folder"
    ET.SubElement(root, "filename").text = basename + ".jpg"
    ET.SubElement(root, "path").text = "path"

    source = ET.SubElement(root, "source")
    ET.SubElement(source, "database").text = "Unknown"

    size = ET.SubElement(root, "size")
    ET.SubElement(size, "width").text = str(w)
    ET.SubElement(size, "height").text = str(h)
    ET.SubElement(size, "depth").text = "3"

    ET.SubElement(root, "segmented").text = "0"

    need_write = False
    for label in labels:
        if label["det"] > 0.2:
            need_write = True
            object = ET.SubElement(root, "object")
            ET.SubElement(object, "name").text = label["name"]
            ET.SubElement(object, "pose").text = "Unspecified"
            ET.SubElement(object, "truncated").text = "0"
            ET.SubElement(object, "difficult").text = "0"
            bndbox = ET.SubElement(object, "bndbox")          
            ET.SubElement(bndbox, "xmin").text = str(label["xmin"])
            ET.SubElement(bndbox, "ymin").text = str(label["ymin"])
            ET.SubElement(bndbox, "xmax").text = str(label["xmax"])
            ET.SubElement(bndbox, "ymax").text = str(label["ymax"])

    if not need_write:
        return
    raw_string = ET.tostring(root, "utf-8")
    reparsed = xml.dom.minidom.parseString(raw_string)
    with open(os.path.join(save_path, basename + ".xml"), "w") as f:
        f.write(reparsed.toprettyxml(indent="\t"))


def main(result_txt, class_i, data_path, save_path):
    os.makedirs(save_path, exist_ok=True)
    all_labels = collect_labels(result_txt, class_i)
    for basename,labels in all_labels.items():
        generate_xml(basename, labels, data_path, save_path)
        print("generated", basename+".xml")

In [ ]:
result_txt = "../../darknet/results/comp4_det_test_HSIL_S.txt"
class_i = "HSIL_S"
data_path = "../../yolo_hsil_s_20181128/test"
save_path = "../../yolo_hsil_s_20181128/test_pred"

main(result_txt, class_i, data_path, save_path)

### generate xmls from prediction, jpg is padded to target size

In [ ]:
def collect_labels(result_txt, class_i):
    all_labels = {}
    with open(result_txt, 'r') as f:
        for line in f.readlines():
            tokens = line.strip().rsplit(' ', 5)  # split from right, in case there is blank space in jpg name
            basename = tokens[0]  # without file extension
            det = float(tokens[1])
            xmin, ymin = int(float(tokens[2])), int(float(tokens[3]))
            xmax, ymax = int(float(tokens[4])), int(float(tokens[5]))
            if not basename in all_labels:
                all_labels[basename] = []
            all_labels[basename].append({"det":det, "xmin":xmin, "ymin":ymin, "xmax":xmax, "ymax":ymax, "name":class_i})
    return all_labels


def generate_xml_with_padding(basename, labels, orig_path, pad_path, save_path):
    jpg_name = os.path.join(orig_path, basename+".jpg")
    with Image.open(jpg_name) as img:
        w, h = img.size
    
    pad_name = os.path.join(pad_path, basename+".txt")
    with open(pad_name, 'r') as f:
        line = f.readline()
        tokens = line.strip().split()
        padx, pady = int(tokens[1]), int(tokens[3])
    
    root = ET.Element("annotation")
    ET.SubElement(root, "folder").text = "folder"
    ET.SubElement(root, "filename").text = basename + ".jpg"
    ET.SubElement(root, "path").text = "path"

    source = ET.SubElement(root, "source")
    ET.SubElement(source, "database").text = "Unknown"

    size = ET.SubElement(root, "size")
    ET.SubElement(size, "width").text = str(w)
    ET.SubElement(size, "height").text = str(h)
    ET.SubElement(size, "depth").text = "3"

    ET.SubElement(root, "segmented").text = "0"

    need_write = False
    for label in labels:
        if label["det"] > 0.2:
            need_write = True
            object = ET.SubElement(root, "object")
            ET.SubElement(object, "name").text = label["name"]
            ET.SubElement(object, "pose").text = "Unspecified"
            ET.SubElement(object, "truncated").text = "0"
            ET.SubElement(object, "difficult").text = "0"
            bndbox = ET.SubElement(object, "bndbox")          
            ET.SubElement(bndbox, "xmin").text = str(label["xmin"]+padx)
            ET.SubElement(bndbox, "ymin").text = str(label["ymin"]+pady)
            ET.SubElement(bndbox, "xmax").text = str(label["xmax"]+padx)
            ET.SubElement(bndbox, "ymax").text = str(label["ymax"]+pady)

    if not need_write:
        return
    raw_string = ET.tostring(root, "utf-8")
    reparsed = xml.dom.minidom.parseString(raw_string)
    with open(os.path.join(save_path, basename + ".xml"), "w") as f:
        f.write(reparsed.toprettyxml(indent="\t"))


def main(result_txt, class_i, orig_path, pad_path, save_path):
    os.makedirs(save_path, exist_ok=True)
    all_labels = collect_labels(result_txt, class_i)
    for basename,labels in all_labels.items():
        generate_xml_with_padding(basename, labels, orig_path, pad_path, save_path)
        print("generated", basename+".xml")

In [ ]:
result_txt = "../../darknet/results/comp4_det_test_HSIL_S.txt"
class_i = "HSIL_S"
orig_path = "../../yolo_hsil_s_20181128/test"
pad_path = "../../yolo_hsil_s_20181128/test608"
save_path = "../../yolo_hsil_s_20181128/test_pred"

main(result_txt, class_i, orig_path, pad_path, save_path)